## Search data

In [ ]:
import os

# Initialize a set to store file paths
parquet_files = set()

# Define the root directory where the Parquet files are located
root_dir = r'C:\Users\guy88\private_file\study_file\DSI321\project\heat_spot_map\df_thai'

# Recursively scan the directory and find all Parquet files
print("🔍 Starting directory scan...")
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.parquet'):
            file_path = os.path.join(root, file)
            parquet_files.add(file_path)  # Add the full path of each .parquet file
            print(f"✅ Found Parquet file: {file_path}")

print("🔍 Scan complete!")


## Shape file

In [4]:
import geopandas as gpd
import matplotlib.pyplot as plt
#adm1 จังหวัด, adm2 อำเภอ
gdf = gpd.read_file(r"C:\Users\guy88\private_file\study_file\DSI321\project\heat_spot_map\shape_file\tha_admbnda_adm1_rtsd_20220121.shp")

## Plot map

### Point map (for test)

In [5]:
def get_fire_color(brightness, min_b=250, max_b=400):
    # Clamp brightness within bounds
    b = max(min_b, min(brightness, max_b))
    ratio = (b - min_b) / (max_b - min_b)

    # Interpolate between orange (255,165,0) and red (255,0,0)
    r = 255
    g = int(165 * (1 - ratio))
    b = 0

    return f'#{r:02X}{g:02X}{b:02X}'


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from datetime import datetime

# ---------- Step 1: Load Thailand province boundaries (ADM1) ----------
shapefile_path = r"C:\Users\guy88\private_file\study_file\DSI321\project\heat_spot_map\shape_file\tha_admbnda_adm1_rtsd_20220121.shp"
gdf = gpd.read_file(shapefile_path)

# Clean datetime columns
gdf = gdf.drop(columns=gdf.select_dtypes(include=['datetime64']).columns)  # Or convert if needed

# Convert to same CRS (Coordinate Reference System)
gdf = gdf.to_crs(epsg=4326)  # EPSG 4326 is latitude/longitude

# ---------- Step 2: Initialize Folium Map ----------
map_center = [13.7367, 100.5231]  # Bangkok
zoom_level = 6
mymap = folium.Map(location=map_center, zoom_start=zoom_level)

# ---------- Step 3: Add Province Boundaries to the Map ----------
folium.GeoJson(
    gdf,
    name='Provinces',
    tooltip=folium.GeoJsonTooltip(fields=['ADM1_TH'], aliases=['จังหวัด']),
    style_function=lambda x: {
        'fillColor': '#add8e6',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.1
    }
).add_to(mymap)

# ---------- Step 4: Add Point Data from Parquet Files ----------
province_fire_count = {}


# Define filter options
filter_mode = 'range'  # Choose 'exact' or 'range'
filter_date_start = datetime.strptime("2025-04-10", "%Y-%m-%d").date()
filter_date_end = datetime.strptime("2025-04-20", "%Y-%m-%d").date()
filter_date_exact = datetime.strptime("2025-04-02", "%Y-%m-%d").date()

for file_path in parquet_files:
    try:
        df = pd.read_parquet(file_path)

        # Ensure 'acq_date' column is datetime.date
        df['acq_date'] = pd.to_datetime(df['acq_date']).dt.date

        # Apply date filtering
        if filter_mode == 'exact':
            df = df[df['acq_date'] == filter_date_exact]
        elif filter_mode == 'range':
            df = df[(df['acq_date'] >= filter_date_start) & (df['acq_date'] <= filter_date_end)]

        if 'latitude' in df.columns and 'longitude' in df.columns and not df.empty:
            for _, row in df.iterrows():
                brightness = row['brightness']
                color = f'#{int(min(255, (brightness - 300) * 1.5)):02x}0000'  # same fading red effect

                folium.CircleMarker(
                    location=[row['latitude'], row['longitude']],
                    radius=3,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=f"🔥 {row['acq_date']}<br>Brightness: {brightness}<br>Confidence: {row['confidence']}"
                ).add_to(mymap)

            print(f"📍 Plotted: {file_path} ({len(df)} points)")
        else:
            print(f"⚠️ No data for selected date(s) in: {file_path}")



    except Exception as e:
        print(f"❌ Failed to process {file_path}: {e}")

# ---------- Step 5: Save the Map ----------
mymap.save("map_with_provinces_and_points.html")
print("✅ Saved map as 'map_with_provinces_and_points.html'")

### HeatMap

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from folium import LinearColormap
from datetime import datetime
import numpy as np

# ---------- Step 1: Load Thailand province boundaries ----------
shapefile_path = r"C:\Users\guy88\private_file\study_file\DSI321\project\heat_spot_map\shape_file\tha_admbnda_adm1_rtsd_20220121.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.drop(columns=gdf.select_dtypes(include=['datetime64']).columns)
gdf = gdf.to_crs(epsg=4326)

# ---------- Step 2: Initialize Folium Map ----------
map_center = [13.7367, 100.5231]  # Bangkok
zoom_level = 6
mymap = folium.Map(location=map_center, zoom_start=zoom_level)

# ---------- Step 3: Add Province Boundaries ----------
folium.GeoJson(
    gdf,
    name='Provinces',
    tooltip=folium.GeoJsonTooltip(fields=['ADM1_TH'], aliases=['จังหวัด']),
    style_function=lambda x: {
        'fillColor': '#e6bead',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1
    }
).add_to(mymap)

# ---------- Step 4: Prepare Fire Points for HeatMap ----------
heat_data = []

# Filter options
filter_mode = 'range'  # Choose 'exact' or 'range'
filter_date_start = datetime.strptime("2025-04-10", "%Y-%m-%d").date()
filter_date_end = datetime.strptime("2025-04-20", "%Y-%m-%d").date()
filter_date_exact = datetime.strptime("2025-04-02", "%Y-%m-%d").date()

# Make sure you define parquet_files as a list of paths
for file_path in parquet_files:
    try:
        df = pd.read_parquet(file_path)
        df['acq_date'] = pd.to_datetime(df['acq_date']).dt.date

        # Filter by date
        if filter_mode == 'exact':
            df = df[df['acq_date'] == filter_date_exact]
        elif filter_mode == 'range':
            df = df[(df['acq_date'] >= filter_date_start) & (df['acq_date'] <= filter_date_end)]

        # Ensure necessary columns exist
        if 'latitude' in df.columns and 'longitude' in df.columns and not df.empty:
            for _, row in df.iterrows():
                lat = row['latitude']
                lon = row['longitude']
                brightness = row['brightness'] if 'brightness' in row else 1  # Optional weight

                # HeatMap supports optional weight as 4th element
                heat_data.append([lat, lon, brightness])  # You can also use confidence if you want

            print(f"🔥 Collected fire points from: {file_path} ({len(df)} points)")
        else:
            print(f"⚠️ No valid fire point data in: {file_path}")

    except Exception as e:
        print(f"❌ Failed to process {file_path}: {e}")

# ---------- Step 5: Add HeatMap ----------
min_b, max_b = 250, 400
normalized_heat_data = []

for lat, lon, brightness in heat_data:
    # Clamp brightness within the range and normalize
    b = max(min_b, min(brightness, max_b))
    weight = (b - min_b) / (max_b - min_b)
    normalized_heat_data.append([lat, lon, weight])

# Define gradient from orange to red
gradient = {
    "0.2": "#FFA500",  # orange
    "0.5": "#FF4500",  # orange-red
    "0.8": "#FF0000",  # red
    "1.0": "#8B0000"   # darkred
}

if normalized_heat_data:
    HeatMap(
        normalized_heat_data,
        radius=10,
        blur=15,
        max_zoom=7,
        gradient=gradient
    ).add_to(mymap)
else:
    print("⚠️ No heat data available to plot.")


# ---------- Step 6: Save Map ----------
mymap.save("map_with_heatmap.html")
print("✅ Saved map as 'map_with_heatmap.html'")